# Meshing

## Inviscid Wedge
![Inviscid Wedge](../assets/inviscid_wedge.png)

https://su2code.github.io/tutorials/Inviscid_Wedge/

# Method 1: Manually define lines and points

In [1]:
from ezmesh import CurveLoop, PlaneSurface, TransfiniteSurfaceField, Geometry, Point, Line, TransfiniteCurveField, visualize_mesh
import gmsh


with Geometry() as geo:
    mesh_size = 0.05

    points = [
        Point([0, 1], mesh_size),
        Point([1.5, 1], mesh_size),
        Point([1.5, 0.2], mesh_size),
        Point([0.5, 0], mesh_size),
        Point([0, 0], mesh_size),
    ]

    segments = [
        Line(start=points[0], end=points[1], label="upper"),
        Line(start=points[1], end=points[2], label="outlet"),
        Line(start=points[2], end=points[3], label="lower"),
        Line(start=points[3], end=points[4], label="lower"),
        Line(start=points[4], end=points[0], label="inlet"),
    ]

    wedge_curve_loop = CurveLoop(segments)
    surface = PlaneSurface([wedge_curve_loop], is_quad_mesh=True)
    gmsh.model.geo.add_point(2,2,2,0.1)

    transfinite_curve_field = TransfiniteCurveField(segments, node_counts=[150,200,100,50,200])
    transfinite_surface_field = TransfiniteSurfaceField.from_labels(surface, labels=("upper", "lower"))

    mesh = geo.generate(surface, fields=[transfinite_curve_field, transfinite_surface_field])
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Overwriting auto display for cadquery Workplane and Shape
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Line)
Info    : Done meshing 1D (Wall 0.000212709s, CPU 0.000221s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Transfinite)
Info    : Done meshing 2D (Wall 0.00222133s, CPU 0.001729s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 5.74999e-06s, CPU 4e-06s)
Info    : 30352 nodes 30706 elements


HTML(value='Coords: ()')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, far=100000.0, near=0.001, position=(0.0, 0.0, 1.0…

,upper
,inlet
,lower
,outlet
,Zone 0


## Method 2: Automatically generate points and lines

In [1]:
from ezmesh import CurveLoop, PlaneSurface, TransfiniteSurfaceField, TransfiniteCurveField, Geometry, visualize_mesh
import numpy as np

with Geometry() as geo:
    wedge_coords = np.array([[0, 1], [1.5, 1], [1.5, 0.2], [0.5, 0], [0, 0]])
    wedge_curve_loop = CurveLoop.from_coords(
        wedge_coords, 
        mesh_size = 0.05,
        group_labels=["upper", "outlet", "lower/1", "lower/2", "inlet"],
    )
    surface = PlaneSurface([wedge_curve_loop], is_quad_mesh=True)

    transfinite_curve_field = TransfiniteCurveField(wedge_curve_loop.edges, node_counts=[150,200,100,50,200])
    transfinite_surface_field = TransfiniteSurfaceField.from_labels(surface, labels=("upper", "lower"))

    mesh = geo.generate(surface, fields=[transfinite_curve_field, transfinite_surface_field])
    visualize_mesh(mesh)
    geo.write("mesh_wedge_inv.su2")

Overwriting auto display for cadquery Workplane and Shape


AttributeError: type object 'TransfiniteSurfaceField' has no attribute 'from_labels'

In [5]:
import cadquery as cq
from ezmesh import GeometryQL


with GeometryQL() as geo:
    geo = (
        geo
        .load(  
            cq.Workplane("XY")
            .polyline([(0, 1), (1.5, 1), (1.5, 0.2), (0.5, 0), (0, 0)])
            .close()
            .extrude(1)
            .faces("<X")
            .transformed(offset=(0.5,0,0))
            .split(keepTop=True)
        )
        .show()
        # .edges("<X")
        # .addPhysicalGroup("inlet")
        # .end()

        # .edges(">X")
        # .addPhysicalGroup("outlet")
        # .end()
        
        # .edges(">Y")
        # .addPhysicalGroup("freestream")
        # .end()

        # .edges("<Y")
        # .addPhysicalGroup("wall")
        # .end()

        # .fromTagged(["edge/1","edge/2"], type="edge")
        # .addPhysicalGroup("wall")
        # .end()

        # .generate()
        # .show("mesh")
    )


AttributeError: 'OCP.gp.gp_Vec' object has no attribute '_address'